In [2]:
from __future__ import print_function
from scipy.io import loadmat
import numpy as np
import tensorflow as tf

/usr/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
tf.logging.set_verbosity(tf.logging.INFO)

In [15]:
SVHN_LOCATION = "/mnt/Data/SVHN/"
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
def preprocessing(img, lbl):
    gray_img = tf.image.rgb_to_grayscale(img)
    tens_location = tf.equal(lbl, 10 * tf.ones(lbl.shape, dtype=tf.uint8))
    tens_with_zeros = tf.where(tens_location, tf.zeros(lbl.shape, dtype=tf.uint8), lbl)
    one_hot = tf.one_hot(tf.reshape(tens_with_zeros, [-1]), 10)
    return gray_img, one_hot

In [16]:
def train_input_fn(train_mat, extra_mat):
    train_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(train_mat['X'], 3, 0), train_mat['y']))
    extra_dataset = tf.data.Dataset.from_tensor_slices((np.moveaxis(extra_mat['X'], 3, 0), extra_mat['y']))

    train_dataset = train_dataset.concatenate(extra_dataset).map(preprocessing).shuffle(10000).batch(
        BATCH_SIZE).repeat(EPOCHS - 1)

    return train_dataset

In [11]:
test_input_fn = lambda test_mat: tf.data.Dataset.from_tensor_slices((np.moveaxis(test_mat['X'], 3, 0),
    test_mat['y'])).map(preprocessing).batch(BATCH_SIZE)

In [ ]:
def resnet_model(features, labels, mode):
    #TODO

In [14]:
print("Loading training data...")
train_mat = loadmat(SVHN_LOCATION + "train_32x32.mat")
extra_mat = loadmat(SVHN_LOCATION + "extra_32x32.mat")
print("Loading testing data...")
test_mat = loadmat(SVHN_LOCATION + "test_32x32.mat")
print("All data loaded")

Loading training data...
Loading testing data...
All data loaded


In [13]:
tensors_to_log = {'accuracy': 'accuracy'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [ ]:
model = tf.estimator.Estimator(model_fn=resnet_model, model_dir='./')

In [ ]:
model.train(input_fn=lambda:train_input_fn(train_mat, extra_mat), hooks=logging_hook)

In [ ]:
print(model.evaluate(input_fn=lambda:test_input_fn(test_mat)))